In [1]:
import torch
import torch.nn.functional as F

from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

In [2]:
Batch_size = 64
transform = transforms.Compose([transforms.Resize((32 ,32)), transforms.ToTensor()])

train_set = datasets.MNIST("data/", train = True, transform = transform, download = True)
test_set = datasets.MNIST("data/", train = False, transform = transform, download = True)

train_loader = DataLoader(train_set, batch_size = Batch_size, shuffle = True)
test_loader = DataLoader(test_set, batch_size = Batch_size, shuffle = False)


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
class LeNet5(nn.Module):
  def __init__(self):
    super().__init__()
    self.Cv1 = nn.Conv2d(1, 6, 5)
    self.Cv2 = nn.Conv2d(6, 16, 5)
    self.Fc1 = nn.Linear(16*5*5, 120)
    self.Fc2 = nn.Linear(120, 84)
    self.Fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.max_pool2d(F.relu(self.Cv1(X)), 2)
    X = F.max_pool2d(F.relu(self.Cv2(X)), 2)
    X = X.view(-1, int(X.nelement() / X.shape[0]))
    X = F.relu(self.Fc1(X))
    X = F.relu(self.Fc2(X))
    X = self.Fc3(X)
    return X

model = LeNet5()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [8]:
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device :', device)
model.to(device)

for epoch in range(epochs):
  epoch_loss = 0

  for i, (X_train, y_train) in enumerate(train_loader):
    X_train = X_train.view((X_train.shape[0], 1, 32, 32)).to(device)

    y_hat = model(X_train)
    loss = criterion(y_hat, y_train.to(device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * len(y_train)

  print("epoch: {:03d}, loss: {:.3f}".format(epoch+1, epoch_loss / len(train_set)))

Device : cuda
epoch: 001, loss: 0.260
epoch: 002, loss: 0.076
epoch: 003, loss: 0.053
epoch: 004, loss: 0.042
epoch: 005, loss: 0.035
epoch: 006, loss: 0.029
epoch: 007, loss: 0.025
epoch: 008, loss: 0.021
epoch: 009, loss: 0.018
epoch: 010, loss: 0.017
epoch: 011, loss: 0.014
epoch: 012, loss: 0.012
epoch: 013, loss: 0.012
epoch: 014, loss: 0.010
epoch: 015, loss: 0.009
epoch: 016, loss: 0.009
epoch: 017, loss: 0.008
epoch: 018, loss: 0.007
epoch: 019, loss: 0.007
epoch: 020, loss: 0.006
epoch: 021, loss: 0.007
epoch: 022, loss: 0.004
epoch: 023, loss: 0.008
epoch: 024, loss: 0.006
epoch: 025, loss: 0.005
epoch: 026, loss: 0.007
epoch: 027, loss: 0.005
epoch: 028, loss: 0.004
epoch: 029, loss: 0.005
epoch: 030, loss: 0.005
epoch: 031, loss: 0.004
epoch: 032, loss: 0.005
epoch: 033, loss: 0.004
epoch: 034, loss: 0.004
epoch: 035, loss: 0.004
epoch: 036, loss: 0.003
epoch: 037, loss: 0.005
epoch: 038, loss: 0.004
epoch: 039, loss: 0.003
epoch: 040, loss: 0.003
epoch: 041, loss: 0.004
ep

In [9]:
y_hat = []

with torch.no_grad():
  model.eval()

  for i, (X_test, y_test) in enumerate(test_loader):
    X_test = X_test.view(X_test.shape[0], 1, 32, 32).to(device)

    scores = model(X_test)
    labels = torch.argmax(scores, dim=1).tolist()

    y_hat.extend(labels)

report = classification_report(test_set.targets, y_hat)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.97      0.99      0.98       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

